<a href="https://colab.research.google.com/github/ARAN1218/UsedcarValuePrediction_AI/blob/main/jupyter%20notebook/6.%E4%B8%AD%E5%8F%A4%E8%BB%8A%E4%BE%A1%E6%A0%BC%E4%BA%88%E6%B8%ACAI(%E4%BE%A1%E6%A0%BC_com)_%E3%83%A2%E3%83%87%E3%83%AB%E4%BD%9C%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%run ./UsedCar_functions.ipynb
import_test()

In [ ]:
import numpy as np
import pandas as pd
import re
from pprint import pprint
from time import sleep

#pandasのデータフレームの行・列表示制限を解除
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

#numpyの表示数制限を解除
np.set_printoptions(threshold=np.inf)


df = pd.read_pickle('Car_Bigdata_Merged.pickle').drop_duplicates()
print(len(df))
df.head(10)

In [ ]:
kkkcom_preprocessing(df)
print(len(df))
df.head(1)

#1000万円以上の車をデータから排除してみる
#df = df[df['総額'] < 1000]
index = df[df['総額'] > 1000].index
df_e = df.copy()
df_e.drop(index, inplace=True)
print(len(df.drop(index)))
print(len(df_e))
#死ぬほど精度が改善された

In [ ]:
#教師データとテストデータに分ける
from sklearn.model_selection import train_test_split

df_l,df_t = train_test_split(df,test_size=0.20,random_state=71,shuffle=True)
print(len(df_l), len(df_t))

In [ ]:
#ラベルエンコーダーの存在を確認
print(len(LE_area.classes_))
print(len(LE_name.classes_))

In [ ]:
#ラベルエンコーダーの要素をpickleファイルで保存
import pickle
#with open('Car_LE_area.pickle', mode='wb') as fp:
    #pickle.dump(LE_area, fp)
#with open('Car_LE_name.pickle', mode='wb') as fp:
    #pickle.dump(LE_name, fp)

In [ ]:
#取り出しテスト
with open('Car_LE_area.pickle', mode='rb') as fp:
    LE_a = pickle.load(fp)
print(LE_a.classes_)

In [ ]:
#説明変数の相関関係を見てみる
import matplotlib.pyplot as plt
%matplotlib inline
import japanize_matplotlib
import seaborn as sns

plt.figure(figsize=(40, 40))
plt.rcParams["font.size"] = 12
sns.heatmap(df.corr(), annot=True, fmt='.2f')
#全長・全幅・全高・重量は全て足す、もしくは消してもいいかも
#車台番号も消す

#エアコン　＋　パワーステアリング　＋　エアバッグ　＋　パワーウィンドウ　＋　キーレスエントリー　＋　ABS　＋　保証フラグ
#CDデッキ　＋　カーナビ　＋　テレビ　＋　ETC
#パワーシート　＋　本革シート　＋　サンルーフ
#3列シート　＋　ウォークスルー　＋　両側スライドドア両側電動

#任意データを扱う際の特徴量の調整

df_erace = df.copy()

eraces = [
    '全長'
    ,'全幅'
    ,'全高'
    ,'車両重量'
    ,'車台番号'
    ,'内装'
    ,'外装'
    ,'鑑定日'
    ,'リサイクル料'
    ,'整備フラグ'
    ,'整備費'
]

for erace in eraces:
    del df_erace[erace]
df_erace.head(1)

In [ ]:
dfdf = pd.concat([df_l, df_t])
print(len(dfdf[dfdf['総額'] > 1000]))
print(len(dfdf[dfdf['総額'] > 900]))
print(len(dfdf[dfdf['総額'] > 800]))
print(len(dfdf[dfdf['総額'] > 700]))
print(len(dfdf[dfdf['総額'] > 600]))
print(len(dfdf[dfdf['総額'] > 500]))
print(len(dfdf[dfdf['総額'] > 400]))
print(len(dfdf[dfdf['総額'] > 300]))
print(len(dfdf[dfdf['総額'] > 200]))
print(len(dfdf[dfdf['総額'] > 100]))
print(len(dfdf))

In [ ]:
#XGBOOST
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

train = df_l.drop(['総額'], axis=1)
target = df_l['総額']
test_x = df_t.drop(['総額'], axis=1)
test_y = df_t['総額']
tr_x,va_x,tr_y,va_y = train_test_split(train, target, test_size=0.20, random_state=71, shuffle=True)
print(tr_x.shape, va_x.shape, tr_y.shape, va_y.shape)

dtrain = xgb.DMatrix(tr_x,label=tr_y)
dvalid = xgb.DMatrix(va_x,label=va_y)
dtest = xgb.DMatrix(test_x)

num_round = 10000
early_stopping_rounds = 250
params = {
    'objective':'reg:squarederror'
    ,'silent':1
    ,'random_state':71
}
watchlist = [(dtrain,'train'),(dvalid,'eval')]
model_xgb = xgb.train(params_xgb
                      ,dtrain
                      ,num_round
                      ,early_stopping_rounds=early_stopping_rounds
                      ,evals=watchlist
                     )
pred = model_xgb.predict(dtest)

display(pd.DataFrame({'総額':test_y, '予測値':pred}))
print('MAE:',mean_absolute_error(test_y,pred))
print('MSE:',mean_squared_error(test_y,pred))
print('RMSE:',np.sqrt(mean_squared_error(test_y,pred)))

MAE: 13.188769603035603
MSE: 704.4652900557686
RMSE: 26.541765013950535

In [ ]:
#クロスバリデーションでモデル評価
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

train = df_l.drop(['総額'], axis=1)
target = df_l['総額']
test_x = df_t.drop(['総額'], axis=1)
test_y = df_t['総額']

mae_final = []
rmse_final = []

for choice in [0.04]:
    score = 0
    mae_scores = []
    rmse_scores = []
    n_splits = 4

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=71)
    for tr_idx, va_idx in kf.split(train):
        tr_x, va_x = train.iloc[tr_idx], train.iloc[va_idx]
        tr_y, va_y = target.iloc[tr_idx], target.iloc[va_idx]

        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dvalid = xgb.DMatrix(va_x, label=va_y)

        num_round = 10000
        early_stopping_rounds= 10 / choice
        params_xgb = {
            'objective':'reg:squarederror'
            ,'max_depth':7
            ,'min_child_weight':1
            ,'colsample_bytree':0.6
            ,'colsample_bylevel':0.9
            ,'subsample':1.0
            ,'gamma':0.7
            ,'eta':choice
            ,'random_state':71
        }
        watchlist = [(dtrain,'train'),(dvalid,'eval')]
        model_xgb_cv = xgb.train(params_xgb
                              ,dtrain
                              ,num_round
                              ,early_stopping_rounds=early_stopping_rounds
                              ,evals=watchlist
                             )

        mae_scores.append(mean_absolute_error(va_y, model_xgb_cv.predict(dvalid)))
        rmse_scores.append(np.sqrt(mean_squared_error(va_y, model_xgb_cv.predict(dvalid))))


    #print('mae_scores:', np.mean(mae_scores))
    #print('rsme_mean_score:', np.mean(rmse_scores))
    mae_final.append(np.mean(mae_scores))
    rmse_final.append(np.mean(rmse_scores))
    
    
print('mae:',mae_final)
print('rmse:',rmse_final)


In [ ]:
[i/10 for i in range(1, 5, 1)]

1:max_depth=11  
mae_scores: 13.90052226444734
rsme_mean_score: 29.11761931911799

2:max_depth=12  
mae_scores: 13.986222617038937
rsme_mean_score: 29.81873635572923

3:max_depth=10  
mae_scores: 13.84769351934502
rsme_mean_score: 28.89949069213789

4:max_depth=9  
mae_scores: 13.726994562852473
rsme_mean_score: 28.616398550169905

5:max_depth=8  
mae_scores: 13.704141782676208
rsme_mean_score: 28.10427173419359

6:max_depth=7  
mae_scores: 13.724626269824387
rsme_mean_score: 27.947585963704896

best_max_depth = 7

mcw  
1
mae_scores: 13.724626269824387
rsme_mean_score: 27.947585963704896

2
mae_scores: 13.73181773923687
rsme_mean_score: 27.97679259660446

3  
mae_scores: 13.842130482613534
rsme_mean_score: 28.31382503828325

4  
mae_scores: 13.833118414156235
rsme_mean_score: 28.169220158876485

5  
mae_scores: 13.855032887814776
rsme_mean_score: 28.02544441242168

0  
mae_scores: 13.724626269824387
rsme_mean_score: 27.947585963704896

best_mcw = 1

bytree  
0.5  
mae: 13.89422047589046
rmse: 27.91681662554806

0.6  
mae_scores: 13.560807028324092
rsme_mean_score: 27.704014078379934

0.7  
mae_scores: 13.533897310546003
rsme_mean_score: 27.870886392046696

0.8  
13.619656501227743
28.006358153286076

0.9  
13.744821258608017
28.064578084316974

best bytree = 0.6

bylevel  
0.6
27.998994508520113

0.7
27.730962055702705

0.8
27.57552498446637

0.9
27.423643009609695

1.0  
13.560807028324092
27.704014078379934

best bylevel = 0.9

subsample  
0.6
29.89845403131721

0.7
29.432146411441273

0.8
28.9629208455904

0.9
28.25420103296127

1.0
27.423643009609695

1.1  
27.423643009609695

best subsample = 1.0

gamma  
0  
13.561457520835786

0.05  
27.558314000272397

0.1  
27.497965077503135

0.2  
27.545711806992568

0.3  
27.5369030902494

0.4  
27.518063687575818

0.5  
27.489749907827374

0.6  
27.554992213544526

0.7  
27.43563970897009

0.8  
27.55549706965453

0.9  
27.486652752597106

best gamma = 0.7

eta  
0.08  
25.640303999652186

0.06  
25.527947230585553

0.04  
25.438190605564394

best eta = 0.04

In [ ]:
10/0.04

In [ ]:
num_round = 10000
early_stopping_rounds= 250
params_xgb = {
            'objective':'reg:squarederror'
            ,'max_depth':7
            ,'min_child_weight':1
            ,'colsample_bytree':0.6
            ,'colsample_bylevel':0.9
            ,'subsample':1.0
            ,'gamma':0.7
            ,'eta':0.04
            ,'random_state':71
        }

チューニング前：  
MAE: 13.048171154501699  
MSE: 680.3767685237565  
RMSE: 26.084032827071745  

チューニング後：  
MAE: 13.177211887619478  
MSE: 689.2265611579014  
RMSE: 26.253124788449497

In [ ]:
#サービス用のモデルを作成

import pickle
#with open('Carmodel_xgb.pickle', mode='wb') as fp:
    #pickle.dump(model_xgb, fp)

In [ ]:
with open('Carmodel_xgb.pickle', mode='rb') as fp:
    model = pickle.load(fp)
    
model.predict(xgb.DMatrix(test_x))

In [ ]:
#ハイパーパラメータチューニング
import optuna
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def opt_xgb(trial):
    params = {
        'objective':'reg:squarederror'
        #,'n_estimators':trial.suggest_int('n_estimators', 0, 1000)
        ,'max_depth':trial.suggest_int('max_depth', 3, 9)
        ,'min_child_weight':trial.suggest_int('min_child_weight', 1, 5)
        ,'subsample':trial.suggest_discrete_uniform('subsample', 0.6, 1.0, 0.1)
        ,'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree', 0.6, 1.0, 0.1)
        ,'colsample_bylevel':trial.suggest_discrete_uniform('colsample_bylevel', 0.6, 1.0, 0.1)
        ,'gamma':trial.suggest_loguniform('gamma', 0.0, 4.0)
        ,'random_state':71
    }
    
    num_round = 20
    early_stopping_rounds=20
    scores = []
    
    kf = KFold(n_splits=4, shuffle=True, random_state=71)
    for tr_idx, va_idx in kf.split(train):
        tr_x, va_x = train.iloc[tr_idx], train.iloc[va_idx]
        tr_y, va_y = target.iloc[tr_idx], target.iloc[va_idx]
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dvalid = xgb.DMatrix(va_x, label=va_y)
        
        watchlist = [(dtrain,'train'),(dvalid,'eval')]
        xgb_tuna = xgb.train(params
                             ,dtrain
                             ,num_round
                             ,early_stopping_rounds=early_stopping_rounds
                             ,evals=watchlist
                            )
        pred_tuna = xgb_tuna.predict(dvalid)
        score = np.sqrt(mean_squared_error(list(va_y), pred_tuna))
        scores.append(score)
        
    return np.mean(scores)

study = optuna.create_study()
study.optimize(opt_xgb
               ,n_trials=100
               #,timeout=300
              )

print(study.best_params,'\n','\n')
print(study.best_value,'\n','\n')
print(study.best_trial,'\n','\n')

In [ ]:
#XGBOOST(パラメータチューニング後テスト)
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

train = df_l.drop(['総額'], axis=1)
target = df_l['総額']
test_x = df_t.drop(['総額'], axis=1)
test_y = df_t['総額']
tr_x,va_x,tr_y,va_y = train_test_split(train, target, test_size=0.20, random_state=71, shuffle=True)
print(tr_x.shape, va_x.shape, tr_y.shape, va_y.shape)

dtrain = xgb.DMatrix(tr_x,label=tr_y)
dvalid = xgb.DMatrix(va_x,label=va_y)
dtest = xgb.DMatrix(test_x)

num_round = 1000
early_stopping_rounds=20
watchlist = [(dtrain,'train'),(dvalid,'eval')]
model_xgb = xgb.train(params_xgb
                      ,dtrain
                      ,num_round
                      ,early_stopping_rounds=early_stopping_rounds
                      ,evals=watchlist
                     )
pred = model_xgb.predict(dtest)

display(pd.DataFrame({'総額':test_y, '予測値':pred}))
print('MAE:',mean_absolute_error(test_y,pred))
print('MSE:',mean_squared_error(test_y,pred))
print('RMSE:',np.sqrt(mean_squared_error(test_y,pred)))

In [ ]:
#LIGHTGBM
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

train = df_l.drop(['総額'], axis=1)
target = df_l['総額']
test_x = df_t.drop(['総額'], axis=1)
test_y = df_t['総額']
tr_x,va_x,tr_y,va_y = train_test_split(train, target, test_size=0.20, random_state=71, shuffle=True)

ltrain = lgb.Dataset(tr_x,tr_y)
lvalid = lgb.Dataset(va_x,va_y)
#ltest = lgb.Dataset(test_x) 予測をする際はDataset型にしていないデータを用いる

#パラメータ準備
params = {'objective':'regression'
          ,'metrics':'rmse'
          ,'silent':1
          ,'random_state':71
         }

#lgb_best_params = study_lgb.best_params
#lgb_best_params['objective'] = 'regression'
#lgb_best_params['metrics'] = 'rmse'
#print(best_params)

num_round = 10000
early_stopping_rounds = 125

#LIGHTGBMモデルに機械学習
model_lgb = lgb.train(params_lgb
                      ,ltrain
                      ,num_boost_round=num_round
                      ,early_stopping_rounds=early_stopping_rounds
                      ,valid_names=['train','valid']
                      ,valid_sets=[ltrain,lvalid]
                     )
pred = model_lgb.predict(test_x) #データを変換せず、そのまま用いる

display(pd.DataFrame({'総額':test_y, '予測値':pred}))
print('MAE:',mean_absolute_error(test_y,pred))
print('MSE:',mean_squared_error(test_y,pred))
print('RMSE:',np.sqrt(mean_squared_error(test_y,pred)))

In [ ]:
#サービス用のモデルを作成

import pickle
#with open('Carmodel_lgb.pickle', mode='wb') as fp:
    #pickle.dump(model_lgb, fp)

In [ ]:
with open('Carmodel_lgb.pickle', mode='rb') as fp:
    model = pickle.load(fp)
    
model.predict(test_x)

In [ ]:
#クロスバリデーションでモデル評価
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

train = df_l.drop(['総額'], axis=1)
target = df_l['総額']
test_x = df_t.drop(['総額'], axis=1)
test_y = df_t['総額']

score = []
scores = []
n_splits = 4

kf = KFold(n_splits=n_splits, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train):
    tr_x, va_x = train.iloc[tr_idx], train.iloc[va_idx]
    tr_y, va_y = target.iloc[tr_idx], target.iloc[va_idx]
    
    ltrain = lgb.Dataset(tr_x, tr_y)
    lvalid = lgb.Dataset(va_x, va_y)
    
    num_round = 10000
    early_stopping_rounds = 10 / 0.08
    params_lgb = {
        'objective':'regression'
        ,'metrics':'rmse'
        ,'num_leaves':25
        ,'max_depth':8
        ,'min_data_in_leaf':10
        ,'bagging_fraction':0.9
        ,'bagging_freq':0
        ,'feature_fraction':0.9
        ,'learning_rate':0.08
        ,'random_state':71
    }
    model_lgb_cv = lgb.train(params_lgb
                          ,ltrain
                          ,num_boost_round=num_round
                          ,early_stopping_rounds=early_stopping_rounds
                          ,valid_names=['train','valid']
                          ,valid_sets=[ltrain,lvalid]
                         )
    
    score.append(model_lgb_cv.predict(va_x))
    scores.append(np.sqrt(mean_squared_error(va_y, model_lgb_cv.predict(va_x))))
    
print('rsme_scores:', scores)
print('rsme_mean_score:', np.mean(scores))


0.05  
27.090037035141165

0.06  
27.123245831268306

0.07  
27.158117678019327

0.08  
27.070208352531992

0.09  
27.182676215866678

0.10  
27.157925769898576

0.12  
27.62


4:num_leaves=31  
27.78884515490398

5:num_leaves=62  
27.811835567021962

6:num_leaves=16  
27.828023152686583

7:num_leaves=40  
27.809208495462922

8:num_leaves=32  
27.836055870397185

9:num_leaves=20  
27.91773047773696

10:num_leaves=25  
27.672723287195247-----

10:num_leaves=26  
27.849112368035268

11:num_leaves=24  
27.718650785083128

10:num_leaves=22  
27.796811594915834

10:num_leaves=  

best_num_leaves = 25

1:max_depth=3  
29.476478231223872

5  
29.476478231223872

7  
27.893220457229365

9  
27.98911154968422

6  
27.927626317533264

8  
27.786565845030943

10  
27.954221727185917

best_max_depth = 8

mdil = 10  
27.507173369572342

9  
27.814311218017103

11  
27.59084952278239

7  
27.568238179352225

best_mdil = 10

b_fraction  
0.8  
27.507173369572342

0.9  
27.507173369572342

1.0  
27.507173369572342

best_b_fraction = 0.9

b_freq  
1
27.51878378307451

2  
27.73655470038359

0  
27.507173369572342

best b_freq = 0

ff  
0.9
27.25832808145858

1.0  
27.507173369572342

0.8  
27.310305373290937

0.85  
27.2862571991923

0.88  
27.49740756827952

0.93  
27.409478038549107

best ff = 0.9

best_score = 27.25832808145858

best e_rate = 0.08

In [ ]:
num_round = 10000
early_stopping_rounds = 125
params_lgb = {
        'objective':'regression'
        ,'metrics':'rmse'
        ,'num_leaves':25
        ,'learning_rate':0.08
        ,'max_depth':8
        ,'min_data_in_leaf':10
        ,'bagging_fraction':0.9
        ,'bagging_freq':0
        ,'feature_fraction':0.9
        ,'random_state':71
    }

チューニング前：27.298125526697884  
チューニング後：26.365860771466078

In [ ]:
#ハイパーパラメータチューニング
import optuna
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def opt_lgb(trial):
    params = {
        'objective':'regression',
        'metrics':'rmse',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'random_state':71,
    }
    
    num_round = 10000
    early_stopping_rounds=50
    
    tr_x,va_x,tr_y,va_y = train_test_split(train, target, test_size=0.20, random_state=71, shuffle=True)
    ltrain = lgb.Dataset(tr_x, label=tr_y)
    lvalid = lgb.Dataset(va_x, label=va_y)
        
    lgb_tuna = lgb.train(params
                        ,ltrain
                        ,num_round
                        ,early_stopping_rounds=early_stopping_rounds
                        ,valid_names=['train','valid']
                        ,valid_sets=[ltrain,lvalid]
                        ,verbose_eval=10
                        )
    pred_lgb_tuna = lgb_tuna.predict(va_x)
    score = np.sqrt(mean_squared_error(list(va_y), pred_lgb_tuna))
        
    return score

study_lgb = optuna.create_study()
study_lgb.optimize(opt_lgb
               ,n_trials=100
               #,timeout=300
              )

print(study_lgb.best_params,'\n','\n')
print(study_lgb.best_value,'\n','\n')
print(study_lgb.best_trial,'\n','\n')

{'lambda_l1': 5.2567979666623315e-08, 'lambda_l2': 1.2323671867671912e-05, 'num_leaves': 39, 'feature_fraction': 0.4467478135070575, 'bagging_fraction': 0.985036426081876, 'bagging_freq': 2, 'min_child_samples': 7} 

In [ ]:
#アンサンブル学習(xgboostとlightgbm)

train = df_l.drop(['総額'], axis=1)
target = df_l['総額']
test_x = df_t.drop(['総額'], axis=1)
test_y = df_t['総額']
tr_x,va_x,tr_y,va_y = train_test_split(train, target, test_size=0.20, random_state=71, shuffle=True)

dtrain = xgb.DMatrix(tr_x, label=tr_y)
dvalid = xgb.DMatrix(va_x, label=va_y)
dtest = xgb.DMatrix(test_x)
ltrain = lgb.Dataset(tr_x, tr_y)
lvalid = lgb.Dataset(va_x, va_y)
ltest = test_x

display(pd.DataFrame({'総額':test_y,
                      'xgb':model_xgb.predict(dtest),
                      'lgb':model_lgb.predict(ltest),
                      'xlgb':(model_xgb.predict(dtest)+model_lgb.predict(ltest))/2
                     }))

print('XGB MAE:',mean_absolute_error(test_y,model_xgb.predict(dtest)))
print('LGB MAE:',mean_absolute_error(test_y,model_lgb.predict(ltest)))
print('ENS MAE:',mean_absolute_error(test_y,(model_xgb.predict(dtest)+model_lgb.predict(ltest))/2))
print()
print('XGB RMSE:',np.sqrt(mean_squared_error(test_y,model_xgb.predict(dtest))))
print('LGB RMSE:',np.sqrt(mean_squared_error(test_y,model_lgb.predict(ltest))))
print('ENS RMSE:',np.sqrt(mean_squared_error(test_y,(model_xgb.predict(dtest)+model_lgb.predict(ltest))/2)))

In [ ]:
#重回帰分析
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

train = df_l.drop(['総額'], axis=1)
target = df_l['総額']
test_x = df_t.drop(['総額'], axis=1)
test_y = df_t['総額']

model_lr = LinearRegression()
model_lr.fit(train, target)

pred = model_lr.predict(test_x)
df_true = pd.DataFrame(list(test_y),columns=['総額'])
df_pred = pd.DataFrame(pred,columns=['予測値'])

print('MAE:',mean_absolute_error(test_y,pred))
print('MSE:',mean_squared_error(test_y,pred))
print('RMSE:',np.sqrt(mean_squared_error(test_y,pred)))
display(pd.concat([df_true,df_pred], axis=1))

In [ ]:
#ランダムフォレスト
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

train = df_l.drop(['総額'], axis=1)
target = df_l['総額']
test_x = df_t.drop(['総額'], axis=1)
test_y = df_t['総額']
tr_x,va_x,tr_y,va_y = train_test_split(train, target, test_size=0.20, random_state=71, shuffle=True)

rf = RandomForestRegressor()
rf.fit(tr_x, tr_y)
pred_rf = rf.predict(test_x)

print('MAE:',mean_absolute_error(test_y,pred_rf))
print('MSE:',mean_squared_error(test_y,pred_rf))
print('RMSE:',np.sqrt(mean_squared_error(test_y,pred_rf)))
display(pd.DataFrame({'総額':test_y, '予測値':pred_rf}))
#xgboostとlightgbmに匹敵する精度(というか、こっちの方が良いかもしれんね)
#ただし、何か時間はかかっている

In [ ]:
#ランダムフォレストモデルによる特徴量の重要度出力

#特徴量の重要度を取得
features = rf.feature_importances_
#特徴量の名前ラベルを取得
label = df_l.columns
#特徴量の重要度順（降順）に並べて表示
indices = np.argsort(features)[::-1]
for i in range(len(features)):
    print(str(i + 1) + "   " + str(label[indices[i]]) + "   " + str(features[indices[i]]))

Index(['車名', '総額', '年式', '走行距離', '地域', '車検', '修復歴', '未使用車', 'ワンオーナー', '禁煙車',
       'ミッション', '駆動方式', '排気量', 'ドア数', '乗車定員', '燃料', 'ハンドル', '記録簿', '燃費',
       '使用燃料', 'エアコン', 'パワーステアリング', 'パワーウィンドウ', 'キーレスエントリー', 'エアバッグ', 'ABS',
       'アルミホイール', '盗難防止システム', 'バックモニターカメラ', 'ベンチシート', 'フルフラット', 'CDデッキ',
       'カーナビ', 'テレビ', 'ETC', '両側スライドドア両側電動', '両側スライドドア', 'スライドドア電動あり',
       'スライドドア', '寒冷地仕様', 'パワーシート', '本革シート', 'サンルーフ', 'ディスチャージドランプ', 'エアロパーツ',
       '3列シート', 'ウォークスルー', 'ローダウン', '福祉車両', '修復', '保証フラグ', '保証期間', '保証距離'],

In [ ]:
df.head(1)